In [1]:
import numpy as np
import tensorflow as tf
from sklearn import preprocessing

In [2]:
raw_csv_data = np.loadtxt('Audiobooks_data.csv',delimiter=',')
unscaled_inputs_all = raw_csv_data[:, 1:-1]
targets_all = raw_csv_data[:,-1]

# Balance the dataset

In [3]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
            
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

## Standardize the inputs

In [4]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

## Shuffle the data

In [5]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

## Split the data into train, validation, test

In [6]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]


print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1790.0 3579 0.5001397038278849
217.0 447 0.4854586129753915
230.0 448 0.5133928571428571


## Save the three datasets in *.npz

In [7]:
np.savez('Audiobook_data_train', inputs= train_inputs, targets= train_targets)
np.savez('Audiobook_data_validation', inputs= validation_inputs, targets= validation_targets)
np.savez('Audiobook_data_test', inputs= train_inputs, targets= train_targets)

## Create the machine learning algorithm

In [8]:
npz = np.load('Audiobook_data_train.npz')

train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobook_data_validation.npz')

validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

npz = np.load('Audiobook_data_test.npz')

test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

## Model

In [10]:
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 100
max_epochs = 100

model.fit(train_inputs, # train inputs
          train_targets, # train targets
          batch_size=batch_size, # batch size
          epochs=max_epochs, # epochs that we will train for (assuming early stopping doesn't kick in)
          validation_data=(validation_inputs, validation_targets), # validation data
          verbose = 2 # making sure we get enough information about the training process
          )  

Epoch 1/100
36/36 - 0s - loss: 0.5631 - accuracy: 0.7835 - val_loss: 0.4468 - val_accuracy: 0.8568
Epoch 2/100
36/36 - 0s - loss: 0.3860 - accuracy: 0.8740 - val_loss: 0.3465 - val_accuracy: 0.8658
Epoch 3/100
36/36 - 0s - loss: 0.3278 - accuracy: 0.8824 - val_loss: 0.3191 - val_accuracy: 0.8747
Epoch 4/100
36/36 - 0s - loss: 0.3044 - accuracy: 0.8885 - val_loss: 0.3056 - val_accuracy: 0.8747
Epoch 5/100
36/36 - 0s - loss: 0.2894 - accuracy: 0.8921 - val_loss: 0.2947 - val_accuracy: 0.8792
Epoch 6/100
36/36 - 0s - loss: 0.2802 - accuracy: 0.8963 - val_loss: 0.2861 - val_accuracy: 0.8926
Epoch 7/100
36/36 - 0s - loss: 0.2706 - accuracy: 0.9016 - val_loss: 0.2778 - val_accuracy: 0.8904
Epoch 8/100
36/36 - 0s - loss: 0.2643 - accuracy: 0.9016 - val_loss: 0.2741 - val_accuracy: 0.8926
Epoch 9/100
36/36 - 0s - loss: 0.2571 - accuracy: 0.9047 - val_loss: 0.2717 - val_accuracy: 0.8926
Epoch 10/100
36/36 - 0s - loss: 0.2528 - accuracy: 0.9058 - val_loss: 0.2742 - val_accuracy: 0.8926
Epoch 11/